In [ ]:
import pandas as pd
import glob
import csv
import pickle
import re
import tweepy
from time import strftime

In [ ]:
file = glob.glob("U:\Twitter Research\Fake News Scraping\Monitoring Accounts\Latest_All\A\A\*.data")

In [ ]:
objects = []
user = []
j=0
for i in file:
    j=j+1
    with (open(i, "rb")) as openfile:
        user.append(i)
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break

In [ ]:
len(objects)

In [ ]:
def tweet_process(alltweets):
    tweet_id = [tweet.id_str for tweet in alltweets]
    tweet_created = [tweet.created_at for tweet in alltweets]
    tweet_text = [tweet.full_text for tweet in alltweets]
    tweet_hashtags = [tweet.entities['hashtags'] for tweet in alltweets]
    user_id = [tweet.user.id for tweet in alltweets]
    screen_name_df = [tweet.user.screen_name for tweet in alltweets]
    retweet_count =  [tweet.retweet_count for tweet in alltweets]
    favorite_count = [tweet.favorite_count for tweet in alltweets]
    lang = [tweet.lang for tweet in alltweets]
    source = [tweet.source for tweet in alltweets]


    retweet_text = []
    media_url = []
    tweet_url = []
    media_type = []
    video_url = []
    
    i=0
    for i in range(len(alltweets)):

        try:
            media_type.append(alltweets[i].extended_entities.get('media')[0].get('type')) 
        except:
            media_type.append(None)
        i = i+1
    
    i=0
    for i in range(len(alltweets)):

        try:
            video_url.append(alltweets[i].extended_entities.get('media')[0].get('video_info').get('variants')[0].get('url')) 
        except:
            video_url.append(None)
        i = i+1
        
        
    
    i=0
    for i in range(len(alltweets)):

        try:
            media_url.append(alltweets[i].entities.get("media")[0].get('media_url_https')) 
        except:
            media_url.append(None)
        i = i+1
        
        
        
    i= 0
    for i in range(len(alltweets)):

        try:
            tweet_url.append(alltweets[i].entities.get('urls')[0].get('expanded_url')) 
        except:
            tweet_url.append(None)
        i = i+1
        
        
        
    i = 0
    for i in range(len(alltweets)):

        try:
            retweet_text.append(alltweets[i].retweeted_status.full_text) 
        except:
            retweet_text.append(None)
        i = i+1


    mod_tweet_hashtags = []
    total_hashtags = []

    for t in tweet_hashtags:
        tag = ""
        count = 0
        if (bool(t) == False):
            mod_tweet_hashtags.append('None')
            total_hashtags.append(count)
        else:
            for i in t:
                tag = tag + " " + i.get('text')
                count +=1
            mod_tweet_hashtags.append(tag)
            total_hashtags.append(count)

    mention = [tweet.entities['user_mentions'] for tweet in alltweets]
    mod_tweet_mention_sc = []
    mod_tweet_mention_id = []
    k = 0
    for t in mention:
        if (bool(t) == False):
            mod_tweet_mention_sc.append('None')
            mod_tweet_mention_id.append('None')
        else:
            mod_tweet_mention_sc.append(mention[k][0]['screen_name'])
            mod_tweet_mention_id.append(mention[k][0]['id_str'])
        k = k+1

    tweet_dict = {'Tweet_ID': tweet_id, 
                  'Tweet_Time': tweet_created, 
                  'Tweet_Text': tweet_text, 
                  'Retweet_Text': retweet_text,
                  'Tweet_HashTags': mod_tweet_hashtags,
                  'Total_Hashtags' : total_hashtags,
                  'Retweet_Count': retweet_count,
                  'Favorite_Count': favorite_count,
                  'Tweet_Source' : source,
                  'Retweet_Screen_Name':mod_tweet_mention_sc,
                  'Retweet_Account_ID':mod_tweet_mention_id,
                  'Tweet_Lang': lang,
                  'User_ID': user_id,
                  'User_ScreenName': screen_name_df, 
                  'Media_URL':media_url,
                  'Media_Type': media_type,
                  'Tweet_URL': tweet_url, 
                  'Video_URL': video_url
                  
                  }  


    dfr = pd.DataFrame(tweet_dict) 
    dfr['Retweet_Source_ScreenName'] = 'None'
    dfr['Retweet'] = False



    for i in range(0, len(dfr)):
        if 'RT' == dfr['Tweet_Text'][i].split()[0]:
            try:
                dfr['Retweet_Source_ScreenName'][i] = dfr['Tweet_Text'][i].split()[1][1:-1]
                dfr['Retweet'][i] = True
            except:
                print(i)
                print(dfr['Tweet_Text'][i])
                dfr['Retweet'][i] = True


    return dfr

In [ ]:
all_data = tweet_process(objects[0])

In [ ]:
for i in range(1, len(objects)):
    temp_data = tweet_process(objects[i])
    frames = [all_data, temp_data]
    all_data = pd.concat(frames)

In [ ]:
all_data = all_data.reset_index(drop = True)

In [ ]:
len(all_data)

In [ ]:
all_data.groupby('User_ID').count()